In [1]:
import pandas as pd
import pickle as pk
from scipy import sparse as sp
from array import array
import numpy as np
import re
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import matplotlib
from matplotlib import pyplot as plt

In [2]:
def fun(s):
    return re.sub(r'([\d]+)','',s).lower()
p_df = pd.read_csv('/Users/richard/Desktop/660D Web Analytics/Final_project/Caesars Palace Hotel&Casino.csv')
doc_complete = p_df['Overall_review']

In [3]:
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

sw = stopwords.words('english')
def stopwords(text):
    '''a function for removing the stopword'''
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)

stemmer = SnowballStemmer("english")

def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

In [4]:
doc_complete=doc_complete.apply(remove_punctuation)
doc_complete2=doc_complete.apply(stopwords)
doc_complete3=doc_complete2.apply(stemming)

In [5]:
list1 = []
for eachline in doc_complete2:
#     print(eachline)
    list1.append(eachline)

In [6]:
newlist111 = []
for i in list1:
    i = i.split(" ")
#     i = i.split()
#     print(i)
    i = [item for item in (filter(lambda x: x.isalpha(),i))]
    #newlist111.append(list111(filter(lambda x : x.isalpha(), i)))
    newlist111.append(i)

In [79]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [80]:
docs = docs_preprocessor(newlist111)

In [106]:
from gensim.models import word2vec
import logging
import pandas as pd
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = word2vec.Word2Vec(docs, min_count=5, size=100, window=5, workers=4 )

2018-04-15 21:40:49,419 : INFO : collecting all words and their counts
2018-04-15 21:40:49,422 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-15 21:40:49,427 : INFO : collected 1199 word types from a corpus of 5231 raw words and 1574 sentences
2018-04-15 21:40:49,431 : INFO : Loading a fresh vocabulary
2018-04-15 21:40:49,435 : INFO : min_count=5 retains 188 unique words (15% of original 1199, drops 1011)
2018-04-15 21:40:49,438 : INFO : min_count=5 leaves 3692 word corpus (70% of original 5231, drops 1539)
2018-04-15 21:40:49,442 : INFO : deleting the raw counts dictionary of 1199 items
2018-04-15 21:40:49,445 : INFO : sample=0.001 downsamples 92 most-common words
2018-04-15 21:40:49,447 : INFO : downsampling leaves estimated 1871 word corpus (50.7% of prior 3692)
2018-04-15 21:40:49,450 : INFO : estimated required memory for 188 words and 100 dimensions: 244400 bytes
2018-04-15 21:40:49,452 : INFO : resetting layer weights
2018-04-15 21:40:49,458 :

In [126]:
print("Top 20 words similar to word 'room'")
print(model.wv.most_similar('room', topn=20))

print("Top 20 words similar to word 'comfort'")
print(model.wv.most_similar(positive=['comfort'],topn=20))

print("Top 20 words similar to word 'location'")
print(model.wv.most_similar('location', topn=20))

print("Top 20 words similar to word 'facility'")
print(model.wv.most_similar(positive=['facility'], negative=['great', 'awesome'], topn=20))

Top 20 words similar to word 'room'
[('palace', 0.496115118265152), ('hotel', 0.4894244968891144), ('location', 0.44084250926971436), ('tower', 0.439990758895874), ('huge', 0.4233834445476532), ('great', 0.4170153737068176), ('extra', 0.4105914831161499), ('staff', 0.4044572710990906), ('like', 0.39149466156959534), ('experience', 0.38411808013916016), ('recommend', 0.3772750794887543), ('definitely', 0.3738331198692322), ('beautiful', 0.3608587980270386), ('shop', 0.3589199483394623), ('stay', 0.35887226462364197), ('free', 0.3466270864009857), ('birthday', 0.33814793825149536), ('busy', 0.33752191066741943), ('back', 0.33265888690948486), ('poor', 0.33139702677726746)]
Top 20 words similar to word 'comfort'
[('first', 0.337268590927124), ('charged', 0.2684023976325989), ('stay', 0.2663932740688324), ('walk', 0.2617397606372833), ('cant', 0.256691575050354), ('thank', 0.25039902329444885), ('iconic', 0.24471451342105865), ('excellent', 0.23729871213436127), ('recommend', 0.23726153373

In [83]:
import csv
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [84]:
sid = SentimentIntensityAnalyzer()
spamreader =p_df['Overall_review']
comments = []
list_ok=[]
for i in spamreader:
    comments.append(i)
#print("Sentiment Analysis for " + filename + ":")
for comment in spamreader:
#         print(comment)
    ss = sid.polarity_scores(comment)
        #print(type(ss))
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]))
        list_ok.append(ss)

compound: -0.5994, 
neg: 0.308, 
neu: 0.692, 
pos: 0.0, 
compound: 0.93, 
neg: 0.0, 
neu: 0.337, 
pos: 0.663, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: -0.34, 
neg: 0.199, 
neu: 0.674, 
pos: 0.127, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.4939, 
neg: 0.0, 
neu: 0.61, 
pos: 0.39, 
compound: 0.6249, 
neg: 0.0, 
neu: 0.661, 
pos: 0.339, 
compound: 0.7688, 
neg: 0.0, 
neu: 0.6, 
pos: 0.4, 
compound: 0.8316, 
neg: 0.0, 
neu: 0.506, 
pos: 0.494, 
compound: -0.0772, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
compound: -0.4939, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
compound: -0.296, 
neg: 0.136, 
neu: 0.864, 
pos: 0.0, 
compound: 0.6892, 
neg: 0.0, 
neu: 0.176, 
pos: 0.824, 
compound: 0.6454, 
neg: 0.0, 
neu: 0.695, 
pos: 0.305, 
compound: 0.2023, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
compound: 0.8316, 
neg: 0.0, 
neu: 0.556, 
pos: 0.444, 
compound: 0.7494, 
neg: 0.211, 
neu:

pos: 0.0, 
compound: 0.9255, 
neg: 0.0, 
neu: 0.342, 
pos: 0.658, 
compound: -0.5216, 
neg: 0.627, 
neu: 0.373, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.6588, 
neg: 0.0, 
neu: 0.532, 
pos: 0.468, 
compound: -0.2023, 
neg: 0.643, 
neu: 0.357, 
pos: 0.0, 
compound: -0.3804, 
neg: 0.393, 
neu: 0.607, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: -0.2755, 
neg: 0.174, 
neu: 0.826, 
pos: 0.0, 
compound: 0.0772, 
neg: 0.34, 
neu: 0.291, 
pos: 0.369, 
compound: 0.4404, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
compound: 0.7615, 
neg: 0.0, 
neu: 0.314, 
pos: 0.686, 
compound: 0.6249, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
compound: 0.2263, 
neg: 0.0, 
neu: 0.725, 
pos: 0.275, 
compound: -0.5962, 
neg: 0.795, 
neu: 0.205, 
pos: 0.0, 
compound: 0.3252, 
neg: 0.0, 
neu: 0.563, 
pos: 0.437, 
compound: 0.6249, 
neg: 0.0, 
neu: 0.423, 
pos: 0.577, 
compound: 0.4926, 
neg: 0.0, 
neu: 0.239, 
pos: 0.761, 
comp

neu: 1.0, 
pos: 0.0, 
compound: 0.7096, 
neg: 0.134, 
neu: 0.403, 
pos: 0.463, 
compound: -0.5994, 
neg: 0.45, 
neu: 0.55, 
pos: 0.0, 
compound: 0.6249, 
neg: 0.0, 
neu: 0.328, 
pos: 0.672, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: -0.3412, 
neg: 0.706, 
neu: 0.294, 
pos: 0.0, 
compound: -0.5334, 
neg: 0.597, 
neu: 0.403, 
pos: 0.0, 
compound: -0.296, 
neg: 0.355, 
neu: 0.645, 
pos: 0.0, 
compound: 0.5994, 
neg: 0.0, 
neu: 0.606, 
pos: 0.394, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: -0.2755, 
neg: 0.15, 
neu: 0.85, 
pos: 0.0, 
compound: 0.765, 
neg: 0.0, 
neu: 0.476, 
pos: 0.524, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.7088, 
neg: 0.0, 
neu: 0.145, 
pos: 0.855, 
compound: -0.3818, 
neg: 0.282, 
neu: 0.544, 
pos: 0.173, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.571

pos: 0.792, 
compound: 0.7717, 
neg: 0.0, 
neu: 0.13, 
pos: 0.87, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.4199, 
neg: 0.0, 
neu: 0.682, 
pos: 0.318, 
compound: 0.0, 
neg: 0.33, 
neu: 0.341, 
pos: 0.33, 
compound: 0.6114, 
neg: 0.0, 
neu: 0.429, 
pos: 0.571, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.3612, 
neg: 0.0, 
neu: 0.667, 
pos: 0.333, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.6369, 
neg: 0.0, 
neu: 0.543, 
pos: 0.457, 
compound: 0.8126, 
neg: 0.0, 
neu: 0.213, 
pos: 0.787, 
compound: 0.5719, 
neg: 0.0, 
neu: 0.213, 
pos: 0.787, 
compound: -0.7351, 
neg: 0.674, 
neu: 0.326, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.3716, 
neg: 0.0, 
neu: 0.44, 
pos: 0.56, 
compound: 0.6588, 
neg: 0.0, 
neu: 0.185, 
pos: 0.815, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.2942, 
neg: 0.0, 
neu:

compound: 0.6249, 
neg: 0.0, 
neu: 0.196, 
pos: 0.804, 
compound: 0.91, 
neg: 0.0, 
neu: 0.286, 
pos: 0.714, 
compound: 0.7579, 
neg: 0.0, 
neu: 0.235, 
pos: 0.765, 
compound: 0.6988, 
neg: 0.0, 
neu: 0.385, 
pos: 0.615, 
compound: 0.34, 
neg: 0.0, 
neu: 0.821, 
pos: 0.179, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: -0.5256, 
neg: 0.772, 
neu: 0.228, 
pos: 0.0, 
compound: 0.636, 
neg: 0.0, 
neu: 0.544, 
pos: 0.456, 
compound: 0.6249, 
neg: 0.0, 
neu: 0.594, 
pos: 0.406, 
compound: 0.2263, 
neg: 0.0, 
neu: 0.725, 
pos: 0.275, 
compound: 0.5106, 
neg: 0.0, 
neu: 0.233, 
pos: 0.767, 
compound: 0.5994, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
compound: 0.9001, 
neg: 0.0, 
neu: 0.2, 
pos: 0.8, 
compound: 0.6588, 
neg: 0.0, 
neu: 0.313, 
pos: 0.687, 
compound: 0.2023, 
neg: 0.167, 
neu: 0.606, 
pos: 0.227, 
compound: 0.2579, 
neg: 0.226, 
neu: 0.407, 
pos: 0.367, 
compound: -0.126, 
neg: 0.199, 
neu: 0.801, 
pos: 0.0, 
compound: 0.7003, 
neg: 0.0, 
neu: 0.463, 
pos: 0.537, 
compoun

In [85]:
ss

{'compound': 0.6361, 'neg': 0.0, 'neu': 0.323, 'pos': 0.677}

In [86]:
j = 0
tmp = pd.DataFrame(list_ok[0],index=[0])
for i in range(3,len(list_ok)):
    if i%4 == 0:
        j += 1
        tmp = pd.concat([tmp,pd.DataFrame(list_ok[i],index = [j])])

In [87]:
df_new=pd.concat([p_df,tmp],axis=1)
df_new.head()

,Unnamed: 0,Overall_review,date,name,nationality,rating,review_times,tag,compound,neg,neu,pos
0,0,booking.com I have used for years no problem t...,"Reviewed: April 3, 2018",James,Ireland,5.8,2 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Octavius P...,-0.5994,0.308,0.692,0.000
1,1,excellent location. Great facilities. Comforta...,"Reviewed: April 1, 2018",Bobbi,United States of America,10.0,1 review,• Leisure trip\n\n\n• Family with young childr...,0.9300,0.000,0.337,0.663
2,2,Nightmare and a rip off,"Reviewed: March 31, 2018",Melanie,Canada,6.7,9 Reviews,• Leisure trip\n\n\n• Family with young childr...,0.0000,0.000,1.000,0.000
3,3,"Loved Vegas, location and facilities, but room...","Reviewed: March 30, 2018",Richard,United Kingdom,7.1,3 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Forum Clas...,-0.3400,0.199,0.674,0.127
4,4,Rough around the edges,"Reviewed: March 30, 2018",Anonymous,Israel,5.0,5 Reviews,• Leisure trip\n\n\n• Family with young childr...,0.0000,0.000,1.000,0.000


In [88]:
df_new.set_index('name')

,Unnamed: 0,Overall_review,date,nationality,rating,review_times,tag,compound,neg,neu,pos
name,,,,,,,,,,,
James,0,booking.com I have used for years no problem t...,"Reviewed: April 3, 2018",Ireland,5.8,2 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Octavius P...,-0.5994,0.308,0.692,0.000
Bobbi,1,excellent location. Great facilities. Comforta...,"Reviewed: April 1, 2018",United States of America,10.0,1 review,• Leisure trip\n\n\n• Family with young childr...,0.9300,0.000,0.337,0.663
Melanie,2,Nightmare and a rip off,"Reviewed: March 31, 2018",Canada,6.7,9 Reviews,• Leisure trip\n\n\n• Family with young childr...,0.0000,0.000,1.000,0.000
Richard,3,"Loved Vegas, location and facilities, but room...","Reviewed: March 30, 2018",United Kingdom,7.1,3 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Forum Clas...,-0.3400,0.199,0.674,0.127
Anonymous,4,Rough around the edges,"Reviewed: March 30, 2018",Israel,5.0,5 Reviews,• Leisure trip\n\n\n• Family with young childr...,0.0000,0.000,1.000,0.000
Stephanie,5,The room was old and stinky❗️,"Reviewed: March 29, 2018",United States of America,5.4,2 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Forum Peti...,0.0000,0.000,1.000,0.000
Judy,6,Vegas BD trip,"Reviewed: March 29, 2018",United States of America,7.5,11 Reviews,• Leisure trip\n\n\n• Travelers with friends\n...,0.0000,0.000,1.000,0.000
Callie,7,Pretty ordinary for the US$900 I paid.,"Reviewed: March 28, 2018",Australia,3.3,3 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Octavius P...,0.4939,0.000,0.610,0.390
Tina,8,Great location for the show we wanted to see.,"Reviewed: March 27, 2018",United States of America,9.2,32 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Forum Clas...,0.6249,0.000,0.661,0.339


In [89]:
df_hh=df_new.drop('Unnamed: 0', axis=1)

In [90]:
df_hh.head()

,Overall_review,date,name,nationality,rating,review_times,tag,compound,neg,neu,pos
0,booking.com I have used for years no problem t...,"Reviewed: April 3, 2018",James,Ireland,5.8,2 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Octavius P...,-0.5994,0.308,0.692,0.000
1,excellent location. Great facilities. Comforta...,"Reviewed: April 1, 2018",Bobbi,United States of America,10.0,1 review,• Leisure trip\n\n\n• Family with young childr...,0.9300,0.000,0.337,0.663
2,Nightmare and a rip off,"Reviewed: March 31, 2018",Melanie,Canada,6.7,9 Reviews,• Leisure trip\n\n\n• Family with young childr...,0.0000,0.000,1.000,0.000
3,"Loved Vegas, location and facilities, but room...","Reviewed: March 30, 2018",Richard,United Kingdom,7.1,3 Reviews,• Leisure trip\n\n\n• Couple\n\n\n• Forum Clas...,-0.3400,0.199,0.674,0.127
4,Rough around the edges,"Reviewed: March 30, 2018",Anonymous,Israel,5.0,5 Reviews,• Leisure trip\n\n\n• Family with young childr...,0.0000,0.000,1.000,0.000


In [91]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction import stop_words

In [92]:
num_of_topics = 100

In [93]:
stops = list(stop_words.ENGLISH_STOP_WORDS) + ['great', 'hotel', 'just', 'like', 've', "'ve", "'m", "'s", "'ll", "ll",
                                               'really', 'place', 'vegas','nice','palace','good', 'wonderful', 'overall'
                                               'excellent', 'amazing','room','best','perfect','overall','caesars', 'stay',
                                              'rooms','staff','excellent','experience']

In [71]:
pipeline = Pipeline([('tfidf', TfidfVectorizer(min_df=0.001, stop_words=stops)),
                     ('tsvd', TruncatedSVD(n_components=num_of_topics, n_iter=10)),
                     ('norm', Normalizer())
                    ])

In [96]:
new_data = pipeline.fit_transform(' '.join(sent) for sent in docs)

In [97]:
tfidf = pipeline.named_steps['tfidf']
tsvd = pipeline.named_steps['tsvd']

In [98]:
tsvd.explained_variance_.sum()

0.51768445065119206

In [99]:
# repurposed from https://de.dariah.eu/tatom/topic_model_python.html#using-non-negative-matrix-factorization
def print_topic_words(components, vocab, num_topics=10, num_of_word_per_topic=10):
    topic_words = []
    vocab_array = np.array(vocab)

    for topic in components:
        word_idx = np.argsort(topic)[::-1][:num_of_word_per_topic]
        topic_words.append((vocab_array[word_idx]).tolist())
        
    for topic, words in list(zip(['Topic_{}'.format(i+1) for i in range(num_topics)], topic_words))[:10]:
        print(topic, words)

In [100]:
print_topic_words(tsvd.components_, tfidf.get_feature_names())

Topic_1 ['location', 'facility', 'vega', 'strip', 'pool', 'clean', 'friendly', 'central', 'fantastic', 'casino']
Topic_2 ['vega', 'caesar', 'time', 'trip', 'love', 'classic', 'central', 'heart', 'luxurious', 'wait']
Topic_3 ['caesar', 'time', 'comfortable', 'hail', 'awesome', 'expensive', 'definitely', 'pool', 'ceasars', 'recommend']
Topic_4 ['time', 'awesome', 'check', 'relaxing', 'staying', 'stayed', 'fabulous', 'birthday', 'ceasars', 'loved']
Topic_5 ['comfortable', 'clean', 'spacious', 'pool', 'beautiful', 'facility', 'friendly', 'bed', 'helpful', 'large']
Topic_6 ['pool', 'expensive', 'fantastic', 'facility', 'swimming', 'view', 'love', 'service', 'area', 'casino']
Topic_7 ['expensive', 'money', 'value', 'disappointing', 'overrated', 'worth', 'lovely', 'luxurious', 'comfortable', 'strip']
Topic_8 ['money', 'worth', 'value', 'awesome', 'dont', 'poor', 'fantastic', 'check', 'service', 'price']
Topic_9 ['awesome', 'come', 'beautiful', 'shower', 'thank', 'expensive', 'relaxing', 'expe